# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [13]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [14]:
import pandas as pd
import numpy as np
import glob

First a function was created to concatenate all the ebola datasets of a given country. This function iterates over all files in the data directory that is provided as a parameter of the function. The function returns a dataframe of all datasets with only the columns provided in column_names parameter which we will need to do this task. We will only need the dates, the description/variable and the totals/national columns.

In [15]:
#a function to collect data from all days recorded
def concatenate(directory_path,column_names):
    directory = glob.glob(directory_path + "/*.csv")
    frame = pd.DataFrame()
    list_ = []
    for file in directory:
        df = pd.read_csv(file,usecols=column_names) # we only need the date, the variable/description and the totals/national for this task
        list_.append(df)
    frame = pd.concat(list_)
    return frame

Then, as shown below the concatenate function was applied to all 3 directories of guinea, liberia and sierra leone.

In [16]:
#importing data
guinea_data=concatenate(DATA_FOLDER + '/ebola/guinea_data',['Date','Description','Totals'])
liberia_data=concatenate(DATA_FOLDER + '/ebola/liberia_data',['Date','Variable','National'])
sl_data=concatenate(DATA_FOLDER + '/ebola/sl_data',['date','variable','National'])

The function below takes a dataset and turns the date column to a column containing only the corresponding month. This function was adapted to all the ways the date was expressed in the datasets. For example the dates in the liberia datasets was expressed as mm/day/year while the other two datasets are written like so : day.month.year or day/month/year.

In [17]:
#take a date written as in our datasets and returns only the month
def dateToMonth(date,liberia) :
    months=['January','February','March','April','May','June',
            'July','August','September','October','November','December']
    if(not(liberia)): 
        return months[int(date.replace('.','-').split('-')[1])-1]
    else : 
        return months[int(date.split('/')[0])-1]

In the code below, all 3 datasets are transformed with the function dateToMonth.

In [18]:
#transform date to month in dataframes
guinea_data['Date']=guinea_data['Date'].apply(lambda x : dateToMonth(x,False))
liberia_data['Date']=liberia_data['Date'].apply(lambda x : dateToMonth(x,True))
sl_data['date']=sl_data['date'].apply(lambda x : dateToMonth(x,False))

Below, all useless rows in guinea data are filtered out and only rows giving new cases and new deaths were kept. Also, the numbers were casted to integers to get them ready to be used for the calculation of the mean. Also, the descriptions were modified to be 'Average daily deaths' and 'Average daily new cases'. 

In [19]:
guinea_data=guinea_data[[description=='Total new cases registered so far' or 
                        description=='New deaths registered today' or description=='New deaths registered' for description in guinea_data['Description'] ]]
guinea_data['Totals']=guinea_data['Totals'].apply(lambda x : int(x))

guinea_data['Description']=guinea_data['Description'].apply(lambda x :'Average daily deaths' 
                                                            if x=='New deaths registered today' or x=='New deaths registered' else
                                                            'Average daily new cases')

Finally, below the mean is calculated by grouping by description and date.

In [20]:
guinea_data=guinea_data.groupby(['Date','Description'])['Totals'].mean()

In [21]:
print(guinea_data)

Date       Description            
August     Average daily deaths        3.4000
           Average daily new cases    25.8000
October    Average daily deaths       15.0000
           Average daily new cases    34.0000
September  Average daily deaths        3.5625
           Average daily new cases    19.6250
Name: Totals, dtype: float64


Same steps as for guinea are performed.

In [22]:
sl_data=sl_data[[description=='new_confirmed' or  description=='death_confirmed' for description in sl_data['variable'] ]]
sl_data=sl_data.dropna()
sl_data['National']=sl_data['National'].apply(lambda x : int(x))
sl_data['variable']=sl_data['variable'].apply(lambda x :'Average daily deaths' 
                                                            if x=='death_confirmed'else
                                                            'Average daily new cases')

In [23]:
sl_data=sl_data.groupby(['date','variable'])['National'].mean()

In [24]:
print(sl_data)

date       variable               
August     Average daily deaths        326.500000
           Average daily new cases      20.611111
December   Average daily deaths       1643.500000
           Average daily new cases      54.333333
November   Average daily deaths       1241.105263
           Average daily new cases      66.210526
October    Average daily deaths        905.464286
           Average daily new cases      56.714286
September  Average daily deaths        476.148148
           Average daily new cases      36.962963
Name: National, dtype: float64


Same steps as for guinea and sierra leone are performed.

In [25]:
liberia_data=liberia_data[[description=='New case/s (confirmed)' or  description=='Newly reported deaths' for description in liberia_data['Variable'] ]]
liberia_data=liberia_data.dropna()
liberia_data['National']=liberia_data['National'].apply(lambda x : int(x))
liberia_data['Variable']=liberia_data['Variable'].apply(lambda x :'Average daily deaths' 
                                                            if x=='Newly reported deaths'else
                                                            'Average daily new cases')

In [26]:
liberia_data=liberia_data.groupby(['Date','Variable'])['National'].mean()

In [27]:
print(liberia_data)

Date       Variable               
August     Average daily deaths         23.222222
           Average daily new cases       5.444444
December   Average daily new cases    2169.375000
July       Average daily deaths          4.272727
           Average daily new cases       1.818182
June       Average daily deaths          2.000000
           Average daily new cases       2.142857
November   Average daily deaths         15.538462
           Average daily new cases       7.800000
October    Average daily deaths         28.040000
           Average daily new cases       1.619048
September  Average daily deaths         37.608696
           Average daily new cases       6.166667
Name: National, dtype: float64


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

First the metadata is read and stored in a dataframe objet.

In [28]:
#Reading the metadata
metadata=pd.read_excel(DATA_FOLDER + "/microbiome/metadata.xls")
metadata['SAMPLE']=metadata['SAMPLE'].apply(lambda x : 'unknown' if str(x)=='nan' else x)

Below, The files MID1,MID2,...,MID9 are read and concatenated into a single dataframe. 
Also, the column called BARCODE is added and the corresponding file name is stored in that column. Meaning that, all rows from MID1's BARCODE column contain 'MID1' and all the rows from MID2's contain 'MID2' and so on. 

In [23]:
MID=[]
def read_and_prepare_file(filename): 
    file=pd.read_excel(DATA_FOLDER + "/microbiome/"+filename+".xls",header=None)
    #add barcode as the first column and contains the filename. 
    file.insert(0,'BARCODE',filename)
    return file
frame=pd.DataFrame()
for i in range(9):
    df=read_and_prepare_file('MID'+str(i+1))
    MID.append(df)
frame=pd.concat(MID)

Below, the dataframe's indices are made unique.

In [24]:
frame = frame.reset_index(drop=True)

Here, a merge is performed using the column 'BARCODE' and therefore giving us the desired resulting dataframe.

In [25]:
frame=pd.merge(frame,metadata,on='BARCODE')

In [26]:
frame

,BARCODE,0,1,GROUP,SAMPLE
0,MID1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,EXTRACTION CONTROL,unknown
1,MID1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,EXTRACTION CONTROL,unknown
2,MID1,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,EXTRACTION CONTROL,unknown
3,MID1,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,EXTRACTION CONTROL,unknown
4,MID1,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,EXTRACTION CONTROL,unknown
5,MID1,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",1,EXTRACTION CONTROL,unknown
6,MID1,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",1,EXTRACTION CONTROL,unknown
7,MID1,"Archaea ""Euryarchaeota"" Archaeoglobi Archaeogl...",1,EXTRACTION CONTROL,unknown
8,MID1,"Archaea ""Euryarchaeota"" Archaeoglobi Archaeogl...",1,EXTRACTION CONTROL,unknown
9,MID1,"Archaea ""Euryarchaeota"" Halobacteria Halobacte...",1,EXTRACTION CONTROL,unknown


## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [22]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here